<a href="https://colab.research.google.com/github/PiotrWrbl/Influence-of-augmented-RNA-seq-gene-expression-data-on-performance-of-disease-classification-model/blob/main/RNA_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [321]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [333]:
sample_info = pd.read_csv("/content/drive/MyDrive/RNA_Project/Data/sampleInfo_Pw.csv", sep="\t")
gene_data = pd.read_csv("/content/drive/MyDrive/RNA_Project/Data/countsAll_fixed_07_07_23.csv", sep="\t")

In [334]:
sample_info = sample_info[['id', 'Group', 'ori_patientgroup', 'isTraining', 'isValidation', 'isTest']]


In [335]:
# sample_info=sample_info[sample_info["ori_patientgroup"].isin(["HC", "ovarianCancer"])]
#sample_info=sample_info[sample_info["ori_patientgroup"].isin(["HC", "ovarianCancer"])]
sample_info=sample_info[sample_info["ori_patientgroup"].isin(["NSCLC"]) | sample_info["Group"].isin(["nonMalignant"])]

In [336]:
sample_info = sample_info.set_index('id')
gene_data_T = gene_data.T
Data = sample_info.join(gene_data_T, how='inner')

In [337]:
Data.head()

,Group,ori_patientgroup,isTraining,isValidation,isTest,ENSG00000000419,ENSG00000000460,ENSG00000000938,ENSG00000001036,ENSG00000001461,...,ENSG00000257267,ENSG00000257923,ENSG00000258890,ENSG00000263563,ENSG00000264538,ENSG00000266356,ENSG00000266714,ENSG00000269028,ENSG00000271043,ENSG00000272168
Vumc-HD-101-TR922,nonMalignant,HC,1,0,0,3.632451,3.775522,4.260973,4.543570,5.029391,...,6.910770,8.159009,4.023624,4.677134,4.677134,5.086967,3.684170,7.911457,6.687347,4.753364
Vumc-HD-49-1-TR1168,nonMalignant,HC,1,0,0,5.574879,3.959200,6.330988,4.304424,4.304424,...,6.388207,7.498989,6.003713,4.765409,3.959200,5.095160,6.075569,6.209074,5.356669,3.959200
Vumc-HD-4-TR1150,nonMalignant,HC,0,1,0,4.280246,4.069523,6.529255,5.316797,4.959521,...,6.242579,7.642872,5.150534,4.851823,5.150534,5.392644,6.281762,6.073907,4.959521,4.453360
Vumc-HD-50-1-TR1169,nonMalignant,HC,0,0,1,5.015662,4.251945,6.531829,4.496862,4.696312,...,6.223482,7.229718,5.084086,3.920578,5.432460,5.666063,5.789113,6.784952,5.380438,4.101956
Vumc-HD-51-1-TR1170,nonMalignant,HC,1,0,0,5.075922,4.360654,6.823066,5.075922,4.543775,...,5.803374,7.811440,5.601746,4.700753,5.672311,4.963115,6.792453,6.628666,5.075922,4.839041


In [338]:
Data['ori_patientgroup'].value_counts()

NSCLC                355
HC                   307
multipleSclerose      86
PulmHyper             72
epilepsy              44
Benign                21
unstableAP            17
chronPancreatitis     16
nSTEMI                15
diabetes              14
Cystic lesion         13
nonsignath            12
stableAP               9
IBD                    6
Hematurie              5
Diverticulitis         3
Name: ori_patientgroup, dtype: int64

In [342]:
# X = Data.drop(['ori_patientgroup'], axis=1)
# y=Data[['ori_patientgroup',"isTraining","isValidation","isTest"]]

X = Data.drop(['ori_patientgroup', 'Group'], axis=1)
y=Data[['Group',"isTraining","isValidation","isTest"]]

In [340]:
y.head()

,Group,isTraining,isValidation,isTest
Vumc-HD-101-TR922,nonMalignant,1,0,0
Vumc-HD-49-1-TR1168,nonMalignant,1,0,0
Vumc-HD-4-TR1150,nonMalignant,0,1,0
Vumc-HD-50-1-TR1169,nonMalignant,0,0,1
Vumc-HD-51-1-TR1170,nonMalignant,1,0,0


In [345]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


X_train = X[(X["isTraining"] == 1) | (X["isValidation"] == 1)]
X_test = X[(X["isTest"] == 1)]
y_train = y[(y["isTraining"] == 1) | (y["isValidation"] == 1)]
y_test = y[(y["isTest"] == 1)]

# X_train = X_train.drop(['isTraining', 'isValidation', 'isTest', 'Group'], axis=1)
# X_test = X_test.drop(['isTraining', 'isValidation', 'isTest', 'Group'], axis=1)
# y_train = y_train.drop(['isTraining', 'isValidation', 'isTest'], axis=1)
# y_test = y_test.drop(['isTraining', 'isValidation', 'isTest'], axis=1)

X_train = X_train.drop(['isTraining', 'isValidation', 'isTest'], axis=1)
X_test = X_test.drop(['isTraining', 'isValidation', 'isTest'], axis=1)
y_train = y_train.drop(['isTraining', 'isValidation', 'isTest'], axis=1)
y_test = y_test.drop(['isTraining', 'isValidation', 'isTest'], axis=1)

le = LabelEncoder()

print(y_train['Group'].value_counts())
print(y_train['Group'].value_counts())

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

nonMalignant    208
Malignant        14
Name: Group, dtype: int64
nonMalignant    208
Malignant        14
Name: Group, dtype: int64


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Linear Regression

In [346]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

scaler = StandardScaler()

X_train_s=scaler.fit_transform(X_train)
X_test_s=scaler.fit_transform(X_test)

classifier = LogisticRegression()

classifier.fit(X_train_s, y_train)

y_pred = classifier.predict(X_test_s)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6666666666666666


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest

In [348]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


scaler = StandardScaler()

X_train_s=scaler.fit_transform(X_train)
X_test_s=scaler.fit_transform(X_test)

forest = RandomForestClassifier()

param_grid = {
    "n_estimators": [50,100,200,300],
    "min_samples_split": [2,4,6,8]

}

grid_search = GridSearchCV(forest, param_grid, cv=5,
                           scoring="accuracy",
                           return_train_score=True)

grid_search.fit(X_train_s, y_train)

best_forest = grid_search.best_estimator_

test_accuracy = best_forest.score(X_test_s, y_test)

In [349]:
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.6846846846846847
